In [3]:
import ipywidgets as ipw
import webbrowser
import requests
import pickle
%run print_map.ipynb

with open('triumphs.data', 'rb') as filehandle:
    triumphs = pickle.load(filehandle)
    
with open('pursuits.data', 'rb') as filehandle:
    pursuits = pickle.load(filehandle)
    
with open('vendors.data', 'rb') as filehandle:
    vendors = pickle.load(filehandle)
    

membershipType = ''
membershipId = ''
characters = {}
active_pursuits = {}
non_instance_pursuits = []
pursuits_for_sale = {}


root = 'https://www.bungie.net/Platform'
clientId = 29004
HEADERS = {'X-API-KEY':'ae7eca0de2cf4bdea9777dc66d862f71'}



#with open('manifest.data', 'rb') as filehandle:
    #manifest = pickle.load(filehandle)  
    


In [7]:
def get_membershipId(gamertag):
    global membershipType
    global membershipId
    
    URL = f'{root}/Destiny2/SearchDestinyPlayer/-1/{gamertag}/'
    r = requests.get(url = URL, headers = HEADERS) 
    data = r.json()
    try:
        membershipId = data['Response'][0]['membershipId']
        membershipType = data['Response'][0]['membershipType']
        return True
    except:
        return False

{'Response': [], 'ErrorCode': 1, 'ThrottleSeconds': 0, 'ErrorStatus': 'Success', 'Message': 'Ok', 'MessageData': {}}
 * None <class 'NoneType'>


In [4]:
def authenticate(code):
        global HEADERS
        body = f'client_id={clientId}&grant_type=authorization_code&code={code}'
        HEADERS2 = {'Content-Type': 'application/x-www-form-urlencoded'}
        URL = f'{root}/App/OAuth/token/'
        r = requests.post(url = URL, data = body, headers = HEADERS2) 
        data = r.json()
        try:
            token = data['access_token']
            HEADERS = {
                'X-API-KEY':'ae7eca0de2cf4bdea9777dc66d862f71',
                'Authorization' : 'bearer ' + token}
            return True
        except:
            return False

In [5]:
def get_characters():
    global characters
    URL = f'{root}/Destiny2/{membershipType}/Profile/{membershipId}/?components=200'
    r = requests.get(url = URL, headers = HEADERS) 
    data = r.json()
    characterIds = list(data['Response']['characters']['data'].keys())
    for characterId in characterIds:
        if data['Response']['characters']['data'][characterId]['classHash'] == 2271682572:
            characters[characterId] = 'Warlock' 
        if data['Response']['characters']['data'][characterId]['classHash'] == 671679327:
            characters[characterId] = 'Hunter' 
        if data['Response']['characters']['data'][characterId]['classHash'] == 3655393761:
            characters[characterId] = 'Titan' 

In [6]:
def refresh_triumphs():
    URL = f'{root}/Destiny2/{membershipType}/Profile/{membershipId}/?components=900'
    r = requests.get(url = URL, headers = HEADERS) 
    data = r.json()
    records = data['Response']['profileRecords']['data']['records']
    for characterId in characters:
        records.update(data['Response']['characterRecords']['data'][characterId]['records'])
    for record in records:
        if int(record) in triumphs:
            if 'objectives' in records[record]:
                obj = 'objectives'
            if 'intervalObjectives' in records[record]:
                obj = 'intervalObjectives'
            triumphs[int(record)]['complete'] = True
            for objective in records[record][obj]:
                objectiveHash = objective['objectiveHash']
                triumphs[int(record)]['objectives'][objectiveHash]['progress'] = objective['progress']
                triumphs[int(record)]['objectives'][objectiveHash]['complete'] = objective['complete']
                if not objective['complete']:
                    triumphs[int(record)]['complete'] = False

In [19]:
def get_pursuits():
    global active_pursuits
    global non_instance_pursuits
    global pursuits_for_sale
    
    active_pursuits = {}
    non_instance_pursuits = []
    pursuits_for_sale = {}
    
    URL = f'{root}/Destiny2/{membershipType}/Profile/{membershipId}/?components=201'
    r  = requests.get(url = URL, headers = HEADERS) 
    data = r.json()
    for characterId in characters:
        for item in data['Response']['characterInventories']['data'][characterId]['items']:
            if item['itemHash'] in pursuits:
                if 'itemInstanceId' in item:
                    if 'vendor' in pursuits[item['itemHash']]:
                        vendor = pursuits[item['itemHash']]['vendor']
                    else:
                        vendor = 'vendor?'
                    p_type = pursuits[item['itemHash']]['type']
                    name = pursuits[item['itemHash']]['name']
                    active_pursuits[item['itemInstanceId']] = {
                        'name' : name,
                        'description' : pursuits[item['itemHash']]['description'],
                        'path' : f'Pursuits/{characters[characterId]}/{p_type}/{vendor}/Inventory/{name}',
                        'objectives' : pursuits[item['itemHash']]['objectives']
                    }
                else:
                    non_instance_pursuits.append(item['itemHash'])
                    
    for characterId in characters:
        URL = f'{root}/Destiny2/{membershipType}/Profile/{membershipId}/Character/{characterId}/Vendors/?components=400,401,402'
        r  = requests.get(url = URL, headers = HEADERS) 
        data = r.json()
        for vendor in data['Response']['sales']['data']:
            vendor_name = vendors[int(vendor)]['name']
            vendor_items = data['Response']['sales']['data'][vendor]['saleItems']
            for item in vendor_items:
                item = vendor_items[item]
                if item['itemHash'] in pursuits:
                    if item['saleStatus'] == 0:
                        p_type = pursuits[item['itemHash']]['type']
                        name = pursuits[item['itemHash']]['name']
                        pursuits_for_sale[str(characterId)+str(item['itemHash'])] = {
                            'name' : name,
                            'description' : pursuits[item['itemHash']]['description'],
                            'path' : f'Pursuits/{characters[characterId]}/{p_type}/{vendor_name}/For Sale/{name}',
                            'objectives' : pursuits[item['itemHash']]['objectives']
                        }

In [52]:
def refresh_pursuits(update_list, progress = 'none'):

    for itemInstanceId in update_list:
        if progress != 'none':
            progress.children[0].value += 100 / len(update_list)
            progress.children[1].value = f'Downloading Data (Pursuit:{itemInstanceId})....'
        URL = f'{root}/Destiny2/{membershipType}/Profile/{membershipId}/Item/{itemInstanceId}/?components=301'
        r = requests.get(url = URL, headers = HEADERS) 
        data = r.json()
        complete = True
        for objective in data['Response']['objectives']['data']['objectives']:
            active_pursuits[itemInstanceId]['objectives'][objective['objectiveHash']]['progress'] = objective['progress']
            active_pursuits[itemInstanceId]['objectives'][objective['objectiveHash']]['complete'] = objective['complete']
            if not objective['complete']:
                complete = False
        active_pursuits[itemInstanceId]['complete'] = complete
